In [1]:
!pip install pretty_midi
!sudo apt install -y fluidsynth
!pip install --upgrade pyfluidsynth

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 19.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 kB 3.6 MB/s eta 0:00:00
  Created wheel for pretty_midi: filename=pretty_midi-0.2.10-py3-none-any.whl size=5592286 sha256=b24c038258f27f6694ee00ba320dedc2e8a1246124980520ba9ae3851e363b9a
  Stored in directory: /root/.cache/pip/wheels/e6/95/ac/15ceaeb2823b04d8e638fd1495357adb8d26c00ccac9d7782e
Successfully built pretty_midi
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  fluid-soundfont-gm libevdev2 libfluidsynth3 libgudev-1.0-0 libinput-bin
  libinput10 libinstpatch-1.0-2 libmd4c0 libmtdev1 libqt5core5a libqt5dbus5
  libqt5gui5 libqt5network5 libqt5svg5 libqt5widgets5 libwacom-bin
  libwacom-common libwacom9 libxcb-icccm4 libxcb-image0 libxcb-keysyms1
  libxcb-render-util0 libxcb-util1 libxcb-xinerama0 libxcb-xin

In [2]:
import numpy as np
import tensorflow as tf
import pandas as pd
import collections
import fluidsynth
import glob
import pretty_midi
from IPython import display
from typing import Dict, List, Optional, Sequence, Tuple


In [5]:
from google.colab import files
uploaded = files.upload()

Saving Dataset-Music-Generation-Using-RNN.zip to Dataset-Music-Generation-Using-RNN (1).zip


In [7]:
!unzip '/content/Dataset-Music-Generation-Using-RNN.zip' -d '/content/music-midi-dataset/'


Archive:  /content/Dataset-Music-Generation-Using-RNN.zip
  inflating: /content/music-midi-dataset/Dataset - Music Generation Using RNN.mid  
  inflating: /content/music-midi-dataset/__MACOSX/._Dataset - Music Generation Using RNN.mid  


In [8]:
!ls /content/music-midi-dataset/


'Dataset - Music Generation Using RNN.mid'   __MACOSX


In [9]:
sampling_rate = 44100

def display_audio(pm, seconds=30):
	waveform = pm.fluidsynth(fs=sampling_rate)
  # Take a sample of the generated waveform to mitigate kernel resets
	waveform_short = waveform[:seconds*sampling_rate]
	return display.Audio(waveform_short, rate=sampling_rate)

pm = pretty_midi.PrettyMIDI()
# Create an instrument instance and add it to the PrettyMIDI object
instrument = pretty_midi.Instrument(program=0, is_drum=False, name='acoustic grand piano')
pm.instruments.append(instrument)
print(pm.instruments)
instrument = pm.instruments[0]

# This code is modified by Susobhan Akhuli


[Instrument(program=0, is_drum=False, name="acoustic grand piano")]


In [12]:
import os
import pretty_midi
import collections
import numpy as np
import pandas as pd

def midi_to_notes(midi_file):
    # Get the absolute path to the MIDI file
    midi_file_path = os.path.join('/content/music-midi-dataset/', midi_file)
    pm = pretty_midi.PrettyMIDI(midi_file_path)
    instrument = pm.instruments[0]
    notes = collections.defaultdict(list)
    sorted_notes = sorted(instrument.notes, key=lambda note: note.start)
    prev_start = sorted_notes[0].start

    for note in sorted_notes:
        start = note.start
        end = note.end
        notes["pitch"].append(note.pitch)
        notes["start"].append(start)
        notes["end"].append(end)
        notes["step"].append(start - prev_start)
        notes["duration"].append(end - start)
        prev_start = start

    return pd.DataFrame({name: np.array(value) for name, value in notes.items()})

# Assuming your midi files are in the 'music-midi-dataset' directory
raw_notes = midi_to_notes('Dataset - Music Generation Using RNN.mid')
note_names = np.vectorize(pretty_midi.note_number_to_name)
sample_note_names = note_names(raw_notes["pitch"])

# This code is modified by Susobhan Akhuli


/usr/local/lib/python3.11/dist-packages/pretty_midi/pretty_midi.py:100: RuntimeWarning: Tempo, Key or Time signature change events found on non-zero tracks.  This is not a valid type 0 or type 1 MIDI file.  Tempo, Key or Time Signature may be wrong.
  warnings.warn(


In [13]:
def notes_to_midi(
  notes: pd.DataFrame,
  out_file: str,
  instrument_name: str,
  velocity: int = 100,  # note loudness
) -> pretty_midi.PrettyMIDI:

  pm = pretty_midi.PrettyMIDI()
  instrument = pretty_midi.Instrument(
      program=pretty_midi.instrument_name_to_program(
          instrument_name))

  prev_start = 0
  for i, note in notes.iterrows():
    start = float(prev_start + note['step'])
    end = float(start + note['duration'])
    note = pretty_midi.Note(
        velocity=velocity,
        pitch=int(note['pitch']),
        start=start,
        end=end,
    )
    instrument.notes.append(note)
    prev_start = start

  pm.instruments.append(instrument)
  pm.write(out_file)
  return pm


In [15]:
import glob
import pandas as pd
import tensorflow as tf

num_files = 5
all_notes = []
filenames = glob.glob('/content/music-midi-dataset/*.mid')  # Ensure the correct path to MIDI files

# Check if filenames is not empty
if not filenames:
    print("No MIDI files found in the specified directory.")

for f in filenames[:num_files]:
    try:
        notes = midi_to_notes(f)
        if not notes.empty:  # Check if notes are not empty
            all_notes.append(notes)
        else:
            print(f"Warning: No notes extracted from {f}")
    except Exception as e:
        print(f"Error processing {f}: {e}")

# Ensure all_notes is not empty before concatenating
if all_notes:
    all_notes = pd.concat(all_notes)
    print(all_notes)
else:
    print("No notes data to concatenate.")

key_order = ["pitch", "step", "duration"]
train_notes = np.stack([all_notes[key] for key in key_order], axis=1)

# Check the shape of train_notes before creating the TensorFlow dataset
print("Train notes shape:", train_notes.shape)

notes_ds = tf.data.Dataset.from_tensor_slices(train_notes)
print(notes_ds.element_spec)


     pitch   start      end   step  duration
0       66   0.000   0.2500  0.000    0.2500
1       66   0.500   0.6250  0.500    0.1250
2       64   0.625   0.7500  0.125    0.1250
3       66   0.750   0.8750  0.125    0.1250
4       66   1.000   1.2500  0.250    0.2500
..     ...     ...      ...    ...       ...
244     69  77.750  77.9375  0.125    0.1875
245     62  78.000  78.1875  0.250    0.1875
246     61  78.500  78.6250  0.500    0.1250
247     61  78.750  78.8750  0.250    0.1250
248     59  79.000  79.1250  0.250    0.1250

[249 rows x 5 columns]
Train notes shape: (249, 3)
TensorSpec(shape=(3,), dtype=tf.float64, name=None)


In [17]:
import tensorflow as tf
import numpy as np

seq_length = 20
vocab_size = 128

def create_sequences(dataset, seq_length, vocab_size=128):
    sequences = []
    targets = []
    num_seq = dataset.shape[0] - seq_length  # Update from train_notes to dataset
    for i in range(num_seq):
        sequence = dataset[i:i+seq_length - 1,:] / [vocab_size, 1 ,1]
        target =  dataset[i+seq_length] / vocab_size
        sequences.append(sequence)
        targets.append(target)

    sequences = np.array(sequences)
    targets = np.array(targets)

    print(sequences.shape, targets.shape)

    # Fixing the dictionary syntax error
    dataset = tf.data.Dataset.from_tensor_slices((sequences, {"pitch": targets[:,0], "step": targets[:,1], "duration": targets[:,2]}))
    return dataset

# Assuming notes_ds is your dataset
seq_ds = create_sequences(train_notes, seq_length=21, vocab_size=vocab_size)

batch_size = 64
buffer_size = 5000

train_ds = seq_ds.shuffle(buffer_size).batch(batch_size)
print(train_ds.element_spec)


(228, 20, 3) (228, 3)
(TensorSpec(shape=(None, 20, 3), dtype=tf.float64, name=None), {'pitch': TensorSpec(shape=(None,), dtype=tf.float64, name=None), 'step': TensorSpec(shape=(None,), dtype=tf.float64, name=None), 'duration': TensorSpec(shape=(None,), dtype=tf.float64, name=None)})


In [19]:
import tensorflow as tf

# Set learning rate and sequence length
learning_rate = 0.005
seq_length = 20

# Define the model architecture
input_data = tf.keras.Input(shape=(seq_length, 3))
x = tf.keras.layers.LSTM(128)(input_data)
outputs = {
    "pitch": tf.keras.layers.Dense(64, name="pitch")(x),
    "step": tf.keras.layers.Dense(1, name="step")(x),
    "duration": tf.keras.layers.Dense(1, name="duration")(x),
}

model = tf.keras.Model(input_data, outputs)

# Define loss functions
loss = {
    "pitch": tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    "step": tf.keras.losses.MeanSquaredError(),
    "duration": tf.keras.losses.MeanSquaredError(),
}

# Set optimizer
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

# Compile the model
model.compile(loss=loss, loss_weights={
        "pitch": 0.05,
        "step": 1.0,
        "duration": 1.0,
    }, optimizer=optimizer)

# Model summary
model.summary()


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 20, 3)     │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ lstm (LSTM)         │ (None, 128)       │     67,584 │ input_layer[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ duration (Dense)    │ (None, 1)         │        129 │ lstm[0][0]        │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ pitch (Dense)       │ (None, 64)        │      8,256 │ lstm[0][0]        │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ step (Dense)        │ (None, 1)         │        129 │ lstm[0][0]        │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 76,098 (297.26 KB)

 Trainable params: 76,098 (297.26 KB)

 Non-trainable params: 0 (0.00 B)

In [21]:
# Train the model
model.fit(train_ds, epochs=10)

# Make predictions on the training dataset
hist = model.predict(train_ds)

# Print the shape of the "duration" output from the predictions
print(hist["duration"].shape)


Epoch 1/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 3s 33ms/step - duration_loss: 0.0013 - loss: 0.2358 - pitch_loss: 4.0814 - step_loss: 0.0289
Epoch 2/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - duration_loss: 1.4180e-04 - loss: 0.1979 - pitch_loss: 3.8122 - step_loss: 0.0065
Epoch 3/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step - duration_loss: 1.7129e-04 - loss: 0.1476 - pitch_loss: 2.7648 - step_loss: 0.0062
Epoch 4/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - duration_loss: 0.1002 - loss: 0.1156 - pitch_loss: 0.1172 - step_loss: 0.0138
Epoch 5/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - duration_loss: 0.0454 - loss: 0.0638 - pitch_loss: 0.0287 - step_loss: 0.0180
Epoch 6/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - duration_loss: 0.0142 - loss: 0.0276 - pitch_loss: 0.0212 - step_loss: 0.0134
Epoch 7/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - duration_loss: 0.0104 - loss: 0.0258 - pitch_loss: 0.0181 - step_loss: 0.0141
Epoch 8/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - duration_loss: 0.0107 - loss: 0.0227 -

In [23]:
def predict_next_note(notes, keras_model, temperature):
    assert temperature > 0
    inputs = np.expand_dims(notes, 0)
    predictions = keras_model.predict(inputs)

    pitch_logits = predictions['pitch']
    step = predictions["step"]
    duration = predictions["duration"]

    # Adjust pitch logits for temperature
    pitch_logits /= temperature

    # Sample from the logits to predict pitch
    pitch = tf.random.categorical(pitch_logits, num_samples=1)
    pitch = tf.squeeze(pitch, axis=-1)

    # Squeeze the step and duration for prediction
    duration = tf.squeeze(duration, axis=-1)
    step = tf.squeeze(step, axis=-1)

    # Make sure step and duration are non-negative
    step = tf.maximum(0, step)
    duration = tf.maximum(0, duration)

    return int(pitch), float(step), float(duration)


temperature = 2.0
num_predictions = 1200

# Sample notes reshaped as in the training process
sample_notes = np.stack([raw_notes[key] for key in key_order], axis=1)

# The initial sequence of notes and pitch is normalized similar to training sequences
input_notes = sample_notes[:seq_length] / np.array([vocab_size, 1, 1])

generated_notes = []
prev_start = 0

for _ in range(num_predictions):
    pitch, step, duration = predict_next_note(input_notes, model, temperature)
    start = prev_start + step
    end = start + duration

    # Append generated note (pitch, step, duration, start, end)
    input_note = (pitch, step, duration)
    generated_notes.append((*input_note, start, end))

    # Update the input sequence for the next prediction
    input_notes = np.delete(input_notes, 0, axis=0)
    input_notes = np.append(input_notes, np.expand_dims(input_note, 0), axis=0)

    prev_start = start

# Convert generated notes to DataFrame for easy processing
generated_notes = pd.DataFrame(
    generated_notes, columns=(*key_order, 'start', 'end')
)

print(generated_notes)


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 823ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 68ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
1/1 ━━━━━━━

In [24]:
out_file = 'gfgmusicgnerate.mid'
instrument_name= pretty_midi.program_to_instrument_name(instrument.program)
out_pm = notes_to_midi(
    generated_notes, out_file=out_file, instrument_name=instrument_name)
display_audio(out_pm , 500)


Output hidden; open in https://colab.research.google.com to view.